In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
import re

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday, USFederalHolidayCalendar
from pandas.tseries.offsets import DateOffset, Day

%matplotlib inline

### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [ ]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
# Константы для использования далее
START_DATE = pd.to_datetime('2013-03-25')
END_DATE = '2020-12-31'
WEEKMASK = 'Mon Tue Wed Thu Fri Sat Sun'

In [59]:
# Читаем производственный календарь
prod_calendar = pd.read_csv(
    r'https://data.gov.ru/sites/default/files/data-20181017t0930-structure-20181017t0930_5.csv'
)
prod_calendar.head()

# Год 2020 отражен не корректно, не учтены ковидные нерабочие дни.

,Год/Месяц,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Всего рабочих дней,Всего праздничных и выходных дней,Количество рабочих часов при 40-часовой рабочей неделе,Количество рабочих часов при 36-часовой рабочей неделе,Количество рабочих часов при 24-часовой рабочей неделе
0,1999,"1,2,3,4,6*,7,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*",251,114,2004,1807.2,1204.8
1,2000,"1,2,3,4,6*,7,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,2,6,7,8*,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,7,11,12,18,19,25,26","2,3,9,10,11*,12,16,17,23,24,30,31",250,116,1995,1800.0,1200.0
2,2001,"1,2,6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25","3,4,7*,8,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29,30*","1,2,5,6,8*,9,12,13,19,20,26,27","2,3,9,10,11*,12,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,6*,7,10,11,17,18,24,25","1,2,8,9,12,15,16,22,23,29,30",251,114,2001,1807.2,1204.8
3,2002,"1,2,5,6,7,12,13,19,20,26,27","2,3,9,10,16,17,22*,23,24,25","2,3,7*,8,9,10,16,17,23,24,30,31","6,7,13,14,20,21,28,30*","1,2,3,4,5,8*,9,10,11,12,19,25,26","1,2,8,9,11*,12,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,6*,7,8,9,16,17,23,24,30","1,7,8,11*,12,13,14,21,22,28,29,31*",250,115,1992,1792.0,1192.0
4,2003,"1,2,3,5*,6,7,11,12,18,19,25,26","1,2,8,9,15,16,22,23,24","1,2,7*,8,9,10,15,16,22,23,29,30","5,6,12,13,19,20,26,27,30*","1,2,3,4,8*,9,10,11,17,18,24,25,31","1,7,8,11*,12,13,14,15,22,28,29","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","4,5,11,12,18,19,25,26","1,2,6*,7,8,9,15,16,22,23,29,30","6,7,11*,12,13,14,20,21,27,28,31*",250,115,1992,1792.0,1192.0


In [61]:
def create_new_nhol(lst, dct):
    '''Создаем список нерабочих дней в 2020 году'''
    for key, value in dct.items():
        for date in value:
            lst.append(Holiday('no work', year=2020, month=int(key), day=date))
    return lst

def get_standard_rules_from_pc(df):
    '''Извлекаем первоначальне даты из производственного календаря'''
    df = df.set_index(['Год/Месяц'])
    df.columns = np.arange(1, 13) 
    df = df.applymap(lambda x: x.split(','))
    return df

def get_pre_holiday(df, lst_nhol):
    '''Читаем полученные даты. Формируем список предпраздничных и рабочих дней'''
    lst_short_bhour = []
    lst_rule = lst_nhol.copy()
    for idx in df.index:
        for col in df.columns:
            for date in df.at[idx, col]:
                if '*' in date[-1]: # находим предпраздничные рабочие дни
                    lst_short_bhour.append(
                        pd.to_datetime(f'{date[0:-1]}.{col}.{idx}')
                    )
                else: # заполняем список выходных и нерабочих дней
                    fill_rule(lst_rule, idx, col, date)
    return lst_short_bhour, lst_rule

def fill_rule(lst, idx, col, date, triger=0):
    '''
    Проверяем на совпадение с нац.праздник. и нераб.днями в 2020 
    и дополняем список выходных
    '''
    date = int(date.split('+')[0])
    for nhol in lst_nhol:
        # ниже if проверяет совпадение с нац.праздником или нерабоч.днем в 2020
        if (col == nhol.month) & (date == nhol.day):
            if idx == 2020:       
                triger = 1  # тригер включается, при наличии совпадения
                break
            elif nhol.name.find('nhol') == 0: # специально созданный признак 
                triger = 1                    # 'nhol' в имени для определения
                break                         # национальных праздников
    if triger == 0: # если нет совпадение с праздником, то дополняем список    
        lst.append(Holiday('weekend', year=idx, month=col, day=date))
    return lst

lst_nhol = [                # список национальных праздников 
    Holiday('nhol Новогодние каникулы', month=1, day=1),
    Holiday('nhol Новогодние каникулы', month=1, day=2),
    Holiday('nhol Новогодние каникулы', month=1, day=3),
    Holiday('nhol Новогодние каникулы', month=1, day=4),
    Holiday('nhol Новогодние каникулы', month=1, day=5),
    Holiday('nhol Новогодние каникулы', month=1, day=6),
    Holiday('nhol Новогодние каникулы', month=1, day=8),
    Holiday('nhol Рождество Христово', month=1, day=7),
    Holiday('nhol День Защитника Отечества', month=2, day=23),
    Holiday('nhol Международный женский день', month=3, day=8),
    Holiday('nhol Праздник Весны и Труда', month=5, day=1),
    Holiday('nhol День Победы', month=5, day=9),
    Holiday('nhol День России', month=6, day=12),
    Holiday('nhol День народного единства', month=11, day=4)
]

dct_2020 = {                 # список нерабочих дней в 2020 году
    '3': [30, 31],
    '4': [date for date in np.arange(1, 31) if date not in [
                                                            4, 5, 11, 12, 
                                                            18, 19, 25, 26
                                                            ]],
    '5': [6, 7, 8],
    '6': [24],
    '7': [1]
}

# создаем список национальных праздников + нерабочих дней 2020 года
lst_new_nhol = create_new_nhol(lst_nhol, dct_2020)

# создаем общий список праздников
df = get_standard_rules_from_pc(prod_calendar.iloc[14:22, 0:13])

%time lst_short_bhour, lst_rule = get_pre_holiday(df, lst_new_nhol)
#lst_rule

CPU times: user 51.9 ms, sys: 0 ns, total: 51.9 ms
Wall time: 51.8 ms


In [62]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    '''Класс-календарь, учитывающий все выходные и праздничные дни'''
    start_date = START_DATE
    end_date = END_DATE
    rules = lst_rule

class RussianBusinessCalendar_nhols(AbstractHolidayCalendar):
    '''Класс-календарь учитывает ТОЛЬКО национальные праздничные дни'''
    start_date = START_DATE
    end_date = END_DATE
    rules = lst_nhol

print(RussianBusinessCalendar)

russian_bdays = pd.offsets.CustomBusinessDay(
    calendar=RussianBusinessCalendar(), weekmask=WEEKMASK
)

type(russian_bdays.holidays)

<class '__main__.RussianBusinessCalendar'>


tuple

In [63]:
# Выявлена ошибка в работе pd.offsets.CustomBusinessMonthEnd
# Если месяц не имеет рабочих дней, то происходит событие ниже
cbme_ru = pd.offsets.CustomBusinessMonthEnd(calendar=RussianBusinessCalendar())

spam_ = pd.to_datetime('2020-03')
for i in np.arange(5):
    spam_ += cbme_ru
    print(spam_) # видно, что нет перехода через месяц, не имеющий рабочих дней

pd.bdate_range(start=START_DATE, end=END_DATE, freq=cbme_ru) # ругается :)

2020-03-27 00:00:00
2020-03-27 00:00:00
2020-03-27 00:00:00
2020-03-27 00:00:00
2020-03-27 00:00:00


ValueError: ignored

In [64]:
def create_df(start_date, end_date):
    '''Создаем DF с временным рядом "1D" в качестве индекса'''
    df = pd.DataFrame(
        index=pd.date_range(start=start_date, end=end_date, freq='1D')
    )
    return df

df_events = create_df(START_DATE, END_DATE)

#df_ = df_events.copy().reset_index()
#print(df_.loc[37 : 48, :])
#print(df_[df_['index'].duplicated(keep=False)])

df_events.head()

        index
37 2013-05-01
38 2013-05-02
39 2013-05-03
40 2013-05-04
41 2013-05-05
42 2013-05-06
43 2013-05-07
44 2013-05-08
45 2013-05-09
46 2013-05-10
47 2013-05-11
48 2013-05-12
Empty DataFrame
Columns: [index]
Index: []


""
2013-03-25
2013-03-26
2013-03-27
2013-03-28
2013-03-29


In [67]:
def fill_df(df, dct, calendar, start_date, end_date, lst_nhol, lst_short_bhour):
    '''Извлекаем ключ и значение словаря, для добавления колонки в DF'''
    for key, value in dct.items():
        df = add_new_col(df, calendar, start_date, end_date, key, value)
#       df_ = df.copy().reset_index()
#       print(df_[df_['index'].duplicated(keep=False)])
    return df   

def add_new_col(df, calendar, start_date, end_date, key, value):
    '''Выбираем функцию, которая будет осуществлять добавление столбца в DF''' 
    if (key == 'выходной') | (key == 'праздник') \
        | (key == 'предпраздничный рабочий день'): 
        return create_col_dti_list(df, key, value)
    elif key == 'последний день месяца':
        return create_col_last_day(df, key, value, start_date, end_date)
    elif (key == 'последний рабочий день месяца') \
        | (key == 'Предновогодний выходной день'):
        return create_col_cbme_error(
            df, key, value, start_date, '2020-03-27', '2020-05', end_date
        )
    elif key == 'день недели':
        return create_cols_period(df, value, df.index.weekday)
    elif key == 'месяц':
        return create_cols_period(df, value, df.index.month)

def create_col_dti_list(df_1, key, value):
    '''Создаем столбец, если на вход подется value DateTimeIndex'''
    df_2 = pd.DataFrame(
        data=[1 for _ in  np.arange(len(value))],
        index=value, columns=[f'{key}']
    )
    return merge_fillna(df_1, df_2)

def create_col_last_day(df_1, key, value, start_date, end_date):
    '''Создаем столбец на основе pd.offsets.MonthEnd'''
    dt_rng = pd.date_range(start=start_date, end=end_date, freq=value)
    df_2 = pd.DataFrame(
        data=[1 for _ in  np.arange(len(dt_rng))],
        index=dt_rng, columns=[f'{key}']
    )
    return merge_fillna(df_1, df_2)

def create_col_cbme_error(
    df_1, key, value, start_date, f_date, s_date, end_date
):
    '''Создаем столбец через pd.offsets.CustomBusinessMonthEnd (с ошибкой)'''
    # ошибку указал в ячейках кода выше
    f_period = pd.date_range(start=start_date, end=f_date, freq=value)
    f_period = pd.DataFrame(index=f_period)
    s_period = pd.date_range(start=s_date, end=end_date, freq=value)
    s_period = pd.DataFrame(index=s_period)
    df_2 = pd.concat([f_period, s_period])
    df_2[f'{key}'] = [1 for _ in np.arange(df_2.shape[0])]
    if key == 'последний рабочий день месяца':
        return merge_fillna(df_1, df_2)
    elif key == 'Предновогодний выходной день':
        df_2.reset_index(inplace=True)
        df_2 = df_2[df_2['index'].dt.month == 12].set_index(['index'])
        return merge_fillna(df_1, df_2)

def create_cols_period(df_1, value, period_to_find):
    '''Создаем DF со стобцом на основе df.index.weekday и df.index.month'''
    df_1['time_period'] = period_to_find
    for period in np.unique(df_1['time_period']):
        df_1[f'{value[period]}'] = np.where(df_1['time_period'] == period, 1, 0)
    df_1.drop(columns=['time_period'], axis=1, inplace=True)
    return df_1

def merge_fillna(df_1, df_2):
    '''Функция merge два DF'''
    # Повторяется часто, поэтому вынесено отдельно
    return df_1.merge(
        df_2, how='left', left_index=True, right_index=True
    ).fillna(0).astype(int)

columns = [        # названия создаваемыемых столбцов
    'выходной', 'праздник', 'предпраздничный рабочий день', 
    'последний день месяца', 'последний рабочий день месяца',
    'пн', 'вт', 'ср', 'чт', 'пт', 'сб',	'вс', 
    'январь',	'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 
    'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь', 
    'Предновогодний выходной день'
]

calendar = RussianBusinessCalendar()  # объект, учитывающий все дни отдыха
calendar_nhol = RussianBusinessCalendar_nhols()  # учитываются только нац.празд.

dct = {           # категории и данные для формирования новых столбцов
    'выходной': calendar.holidays(start=START_DATE, end=END_DATE),
    'праздник': calendar_nhol.holidays(start=START_DATE, end=END_DATE),
    'предпраздничный рабочий день': lst_short_bhour,
    'последний день месяца': pd.offsets.MonthEnd(),
    'последний рабочий день месяца': pd.offsets.CustomBusinessMonthEnd(
        calendar=calendar
    ),
    'день недели': columns[5:12],
    'месяц': columns[11:24],
    'Предновогодний выходной день': pd.offsets.CustomBusinessMonthEnd(
        calendar=calendar
    )
}

df = fill_df(
    df_events, dct, calendar, START_DATE, END_DATE, lst_nhol, lst_short_bhour
)

df.head(10)

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2013-03-25,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-26,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-27,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-28,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-29,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-30,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-31,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-04-01,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2013-04-02,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2013-04-03,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [69]:
df_sber = pd.read_csv(
    './sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
    index_col='TRADEDATE', parse_dates=True
)

df_sber.head()

,CLOSE
TRADEDATE,
2013-03-25,98.79
2013-03-26,97.20
2013-03-27,96.75
2013-03-28,98.59
2013-03-29,98.76


In [71]:
# Определяем диапазоны дат для слияния двух DF
start_date = min(df_sber.index)
end_date = max(df_sber.index)
# сокращаем диапазон дат в DF_с_dummy календарем
df_ = df[(df.index >= start_date) & (df.index <= end_date)]
# Делаем слияние DF_с_котировками c DF_с_dummy календарем
# ниже код позволяет обойтись без df.resample('1D').asfreq() или df.asfreq('1D') 
df_ = df_sber.merge(df_, how='right', left_index=True, right_index=True) 
df_.head()

,CLOSE,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2013-03-25,98.79,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-26,97.20,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-27,96.75,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-28,98.59,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-03-29,98.76,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [72]:
def create_lags(df_1, lst_lags, freq):
    '''Создаем лаговые переменные рабочих дней с глубинами: 1, 3, 5'''
    df_ = pd.DataFrame(index=df_1.index)
    for lag in lst_lags:
        sr = df_1['CLOSE'].copy()
        sr.name = f'CLOSE_lag_{lag}' # переименуем столбцец, чтобы не ругался            
        sr = sr.shift(lag, freq=freq) # наличие np.NaN вызывает затроение дат
        sr = sr.loc[~sr.index.duplicated(keep='first')] # удаляем затроения
        df_ = df_.merge(sr, how='left', left_index=True, right_index=True)       
    return df_1.merge(df_, how='left', left_index=True, right_index=True)

freq = pd.offsets.CustomBusinessDay(   # offset для задания freq в shift().
    calendar=RussianBusinessCalendar(), weekmask=WEEKMASK 
)

lst_lags = [1, 3, 5]

df_shift = create_lags(df_, lst_lags, freq)
df_shift.tail(15)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:694: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


,CLOSE,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день,CLOSE_lag_1,CLOSE_lag_3,CLOSE_lag_5
2019-12-27,252.06,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,248.24,248.67,244.71
2019-12-28,NaN,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN
2019-12-29,NaN,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,NaN,NaN
2019-12-30,254.75,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,252.06,248.04,248.80
2019-12-31,NaN,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,254.75,248.24,248.67
2020-01-01,NaN,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2020-01-02,NaN,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2020-01-03,255.00,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2020-01-04,NaN,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2020-01-05,NaN,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [75]:
def create_rolling(df_1, windows, lst_days, lst_agg, lst_cols):
    '''
    Скользящие статистики ['mean', 'median', 'max', 'min', 'std'] 
    с окнами 1, 3, 5 сгрупированные по дням недели
    '''
    for window in windows:
        df_tmp = pd.DataFrame(columns=lst_cols)
        for day_number, day_name in enumerate(lst_days): # проход по дням недели
            df_ = df_1['CLOSE'].resample(
                pd.offsets.Week(weekday=day_number)
            ).interpolate().rolling(window=window).agg(lst_agg).shift(1)
            df_tmp = pd.concat([df_tmp, df_])
        df_1 = df_1.merge(
            df_tmp, how='left', left_index=True, 
            right_index=True, suffixes=('', f'_{window}') # суфикс, чтобы различ
        )
    return df_1

windows = [1, 3, 5]   # окна для скользящих
lst_agg = [np.mean, np.median, np.max, np.min, np.std]  # вычисляемые статистики
lst_cols = ['mean', 'median', 'amax', 'amin', 'std'] # названия столбцов

df_rolling = create_rolling(df_shift, windows, columns[5:12], lst_agg, lst_cols)
df_rolling

,CLOSE,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день,CLOSE_lag_1,CLOSE_lag_3,CLOSE_lag_5,mean,median,amax,amin,std,mean_3,median_3,amax_3,amin_3,std_3,mean_5,median_5,amax_5,amin_5,std_5
2013-03-25,98.79,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-26,97.20,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,98.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-27,96.75,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,97.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-28,98.59,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,96.75,98.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-29,98.76,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,98.59,97.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06,253.90,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,254.750,254.750,254.750,254.750,NaN,248.333333,248.80,254.750,241.45,6.662269,243.162,241.45,254.750,233.77,8.583244
2020-01-07,NaN,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,248.670,248.670,248.670,248.670,NaN,246.313333,248.67,248.670,241.60,4.081866,241.502,241.60,248.670,230.66,7.635795
2020-01-08,259.15,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,253.595,253.595,253.595,253.595,NaN,249.278333,248.04,253.595,246.20,3.849884,243.949,246.20,253.595,231.59,8.377994
2020-01-09,257.99,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,252.06,248.04,253.115,253.115,253.115,253.115,NaN,248.451667,248.24,253.115,244.00,4.561185,243.623,244.00,253.115,231.71,8.063758


In [78]:
def create_expanding(df_1, lst_cols, lst_agg, dct_agg):
    '''
    Статистики ['mean', 'median', 'max', 'min', 'std'] 
    в расширяющемся окне сгруппированные 
    по рабочим, праздничным и предпраздничным дням
    '''
    df = df_1['CLOSE'].copy()
    for cat, val in dct_agg.items():
        if cat == 'рабочий':
            df_2 = df.resample(val).interpolate().expanding().agg(lst_agg)
            df_1 = merge_agg(df_1, df_2, cat)
        else:
            df_2 = pd.DataFrame(df).merge(
                pd.DataFrame(index=val), how='inner', 
                left_index=True, right_index=True
            )            
            df_2 = df_2.expanding().agg(lst_agg)
            df_1 = merge_agg(df_1, df_2, cat)
    return df_1

def merge_agg(df_1, df_2, cat):
    '''Слияние DF'''
    # Функция вынесена отдельно, потому что встречается 2 раза
    return df_1.merge(
        df_2, how='left', left_index=True, right_index=True, 
        suffixes=['', f'_{cat}']
    )

lst_cols = ['mean', 'median', 'amax', 'amin', 'std']
lst_agg = [np.mean, np.median, np.max, np.min, np.std]
# календарь с категориями дней и наборами дат для выполнения агрегации
dct_agg = {   
    'рабочий': pd.offsets.CustomBusinessDay(
    calendar=RussianBusinessCalendar(), weekmask=WEEKMASK
    ),
    'праздник': RussianBusinessCalendar_nhols().holidays(
        start=START_DATE, end=END_DATE
    ),
    'предпраздник': lst_short_bhour
}

df_expanding = create_expanding(df_rolling, lst_cols, lst_agg, dct_agg)
df_expanding.tail(11)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,CLOSE,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день,CLOSE_lag_1,CLOSE_lag_3,CLOSE_lag_5,mean,median,amax,amin,std,mean_3,median_3,amax_3,amin_3,std_3,mean_5,median_5,amax_5,amin_5,std_5,mean_рабочий,median_рабочий,amax_рабочий,amin_рабочий,std_рабочий,"('CLOSE', 'mean')","('CLOSE', 'median')","('CLOSE', 'amax')","('CLOSE', 'amin')","('CLOSE', 'std')","(CLOSE_предпраздник, mean)","(CLOSE_предпраздник, median)","(CLOSE_предпраздник, amax)","(CLOSE_предпраздник, amin)","(CLOSE_предпраздник, std)"
2019-12-31,NaN,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,254.75,248.24,248.67,248.670,248.670,248.670,248.670,NaN,242.726667,241.60,248.670,237.91,5.467763,238.768,237.91,248.670,230.66,6.832706,145.591294,139.500,279.97,53.5,61.870146,NaN,NaN,NaN,NaN,NaN,144.101538,105.925,277.49,70.68,68.108385
2020-01-01,NaN,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,248.040,248.040,248.040,248.040,NaN,244.853333,246.20,248.040,240.32,4.032336,240.074,240.32,248.040,231.59,7.198839,NaN,NaN,NaN,NaN,NaN,147.132222,171.295,238.80,56.37,62.897050,NaN,NaN,NaN,NaN,NaN
2020-01-02,NaN,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,248.240,248.240,248.240,248.240,NaN,244.430000,244.00,248.240,241.05,3.614236,240.040,241.05,248.240,231.71,6.647484,NaN,NaN,NaN,NaN,NaN,147.132222,171.295,238.80,56.37,62.897050,NaN,NaN,NaN,NaN,NaN
2020-01-03,255.00,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,252.060,252.060,252.060,252.060,NaN,245.993333,244.71,252.060,241.21,5.537674,241.420,241.21,252.060,233.98,7.397124,NaN,NaN,NaN,NaN,NaN,152.809474,171.900,255.00,56.37,65.944305,NaN,NaN,NaN,NaN,NaN
2020-01-04,NaN,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,186.300,186.300,186.300,186.300,NaN,186.300000,186.30,186.300,186.30,0.000000,186.300,186.30,186.300,186.30,0.000000,NaN,NaN,NaN,NaN,NaN,152.809474,171.900,255.00,56.37,65.944305,NaN,NaN,NaN,NaN,NaN
2020-01-05,NaN,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.809474,171.900,255.00,56.37,65.944305,NaN,NaN,NaN,NaN,NaN
2020-01-06,253.90,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,254.750,254.750,254.750,254.750,NaN,248.333333,248.80,254.750,241.45,6.662269,243.162,241.45,254.750,233.77,8.583244,NaN,NaN,NaN,NaN,NaN,157.864000,173.925,255.00,56.37,68.049539,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,248.670,248.670,248.670,248.670,NaN,246.313333,248.67,248.670,241.60,4.081866,241.502,241.60,248.670,230.66,7.635795,NaN,NaN,NaN,NaN,NaN,157.864000,173.925,255.00,56.37,68.049539,NaN,NaN,NaN,NaN,NaN
2020-01-08,259.15,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,253.595,253.595,253.595,253.595,NaN,249.278333,248.04,253.595,246.20,3.849884,243.949,246.20,253.595,231.59,8.377994,NaN,NaN,NaN,NaN,NaN,162.687143,175.950,259.15,56.37,69.912228,NaN,NaN,NaN,NaN,NaN
2020-01-09,257.99,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,252.06,248.04,253.115,253.115,253.115,253.115,NaN,248.451667,248.24,253.115,244.00,4.561185,243.623,244.00,253.115,231.71,8.063758,145.658278,139.555,279.97,53.5,61.912529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# просто код на память, красивое решение
cal = pd.tseries.holiday.get_calendar('RussianBusinessCalendar')

cal.rules[0:5]

[Holiday: nhol Новогодние каникулы (month=1, day=1, ),
 Holiday: nhol Новогодние каникулы (month=1, day=2, ),
 Holiday: nhol Новогодние каникулы (month=1, day=3, ),
 Holiday: nhol Новогодние каникулы (month=1, day=4, ),
 Holiday: nhol Новогодние каникулы (month=1, day=5, )]

In [ ]:
# код на память
calendar = RussianBusinessCalendar()
print(calendar.holidays(start=START_DATE, end=END_DATE))
print(type(calendar.holidays(start=START_DATE, end=END_DATE)))

DatetimeIndex(['2013-05-01', '2013-05-02', '2013-05-03', '2013-05-04',
               '2013-05-05', '2013-05-09', '2013-05-10', '2013-05-11',
               '2013-05-12', '2013-05-18',
               ...
               '2020-11-28', '2020-11-29', '2020-12-05', '2020-12-06',
               '2020-12-12', '2020-12-13', '2020-12-19', '2020-12-20',
               '2020-12-26', '2020-12-27'],
              dtype='datetime64[ns]', length=879, freq=None)
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
